In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ

In [3]:
feedback_bits = 64
ncodebooks=512
ncentroids=128
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test_f%i.npy' % feedback_bits
    featurepath_test = 'feature_test_f%i.npy' % feedback_bits
    y_test = 'y_test_f%i.npy' % feedback_bits
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw_f%i.npy' % feedback_bits
biaspath = 'encoder_fcb_f%i.npy' % feedback_bits
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, "fc1")
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 


In [5]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Mithral


In [6]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [7]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[ 0.32216886  0.09217934  0.35763702 ...  0.20204398 -0.19147566
  -0.15363818]
 [-0.05356499  0.6557801  -0.26858604 ...  0.6404001  -0.06623106
  -0.65461665]
 [-0.11618729  0.09217934  0.8586155  ...  0.95351166  0.30950278
   0.3473403 ]
 ...
 [ 0.38479117 -0.03306528 -0.14334145 ...  0.7030224   0.9983482
  -0.46674973]
 [-0.6171658  -0.09568759 -0.26858604 ... -0.11106756  0.05901356
  -0.09101588]
 [ 0.44741347  0.40529087 -0.33120838 ... -0.799913   -0.25409797
   0.4099626 ]]
(32000, 64)
